In [10]:
import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import plotly.graph_objs as go
import pandas as pd

import random
import pandas as pd
from faker import Faker
from datetime import datetime, timedelta
import numpy as np  # Import numpy for NaN

fake = Faker()

# Define product categories and representative product names
products = {
    "Electronics": ["Smartphone", "Laptop", "Headphones", "Camera", "Smartwatch"],
    "Fashion": ["T-shirt", "Jeans", "Sneakers", "Jacket", "Dress"],
    "Home & Kitchen": ["Blender", "Vacuum Cleaner", "Coffee Maker", "Microwave Oven", "Dishwasher"],
    "Books": ["Novel", "Biography", "Science Fiction", "Non-Fiction", "Fantasy"],
    "Sports & Outdoors": ["Tennis Racket", "Yoga Mat", "Dumbbells", "Bicycle", "Running Shoes"],
    "Beauty & Personal Care": ["Shampoo", "Lipstick", "Moisturizer", "Perfume", "Hair Dryer"],
    "Toys & Games": ["Action Figure", "Board Game", "Puzzle", "Lego Set", "Doll"],
    "Grocery": ["Organic Milk", "Eggs", "Bread", "Chicken Breast", "Rice"],
    "Automotive": ["Car Cover", "Engine Oil", "Car Vacuum", "Tire Inflator", "Car Battery"],
    "Health & Household": ["Vitamins", "Toothpaste", "Laundry Detergent", "Disinfectant Wipes", "Toilet Paper"]
}

# Define payment methods
payment_methods = [
    "Amazon Pay UPI", "Google Pay", "PhonePe", "Credit Card", 
    "Debit Card", "EMI", "Net Banking", "Cash on Delivery"
]

# Generate random data for 50 orders
data = []

# Generate a random date within the past two years
def random_date():
    end_date = datetime.now()
    start_date = end_date - timedelta(days=730)  # Approximately two years
    return fake.date_between(start_date=start_date, end_date=end_date)

for order_id in range(1, 51):  # Change to 51 to generate 50 orders (1 to 50)
    product_category = random.choice(list(products.keys()))
    product_name = random.choice(products[product_category])
    mrp = round(random.uniform(100, 20000), 2)  # MRP between 100 and 2000
    price_for_customer = round(mrp * random.uniform(0.5, 1.0), 2)  # Price between 50% and 100% of MRP
    savings = mrp - price_for_customer
    payment_method = random.choice(payment_methods)
    order_date = random_date()
    order_month = order_date.strftime("%B")
    
    if payment_method == "EMI":
        # For EMI, generate random monthly payment amount and duration
        monthly_payment = round(price_for_customer / random.randint(3, 12), 2)  # Divide total amount by 3 to 12 months
        duration_months = random.randint(3, 12)
    else:
        # For other payment methods, set monthly_payment and duration to NaN
        monthly_payment = np.nan
        duration_months = np.nan
    
    data.append([
        order_id, product_name, product_category, mrp, price_for_customer, savings, 
        payment_method, monthly_payment, duration_months, order_date, order_month
    ])

# Create a DataFrame
columns = ["Order ID", "Product Name", "Product Category", "MRP", "Price for Customer", 
           "Savings", "Payment Method", "Monthly Payment", "Duration (months)", "Order Date", "Order Month"]

df = pd.DataFrame(data, columns=columns)

# Display the DataFrame
# print(df)

# Optionally, save to a CSV file
df.to_csv("customer_orders.csv", index=False)


In [7]:

def plot_spending_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_spending = filtered_df.groupby('Product Category')['Price for Customer'].sum().reset_index()
    spending_figure = {
        'data': [
            {'y': category_spending['Product Category'].tolist(), 'x': category_spending['Price for Customer'].tolist(), 'type': 'bar', 'orientation': 'h', 'name': 'Spending'}
        ],
        'layout': {
            'title': 'Spending by Category',
            'yaxis': {'title': 'Category', 'automargin': True},
            'xaxis': {'title': 'Total Spending'},
            'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}
        }
    }
    return spending_figure

def plot_savings_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_savings = filtered_df.groupby('Product Category')['Savings'].sum().reset_index()
    savings_figure = {
        'data': [
            {'y': category_savings['Product Category'].tolist(), 'x': category_savings['Savings'].tolist(), 'type': 'bar', 'orientation': 'h', 'name': 'Savings'}
        ],
        'layout': {
            'title': 'Savings by Category',
            'yaxis': {'title': 'Category', 'automargin': True},
            'xaxis': {'title': 'Total Savings'},
            'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}
        }
    }
    return savings_figure

def create_total_spending_card(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    total_spending = filtered_df['Price for Customer'].sum()
    card = {
        'title': 'Total Spending',
        'value': f'₹{total_spending:,.2f}'
    }
    return card

def create_total_savings_card(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    total_savings = filtered_df['Savings'].sum()
    card = {
        'title': 'Total Savings',
        'value': f'₹{total_savings:,.2f}'
    }
    return card

# Example usage:
filtered_df = df.copy()  # Load your filtered DataFrame here
selected_year = 'Overall'
selected_month = 'Overall'

spending_fig = plot_spending_by_category(filtered_df, selected_year, selected_month)
savings_fig = plot_savings_by_category(filtered_df, selected_year, selected_month)
total_spending_card = create_total_spending_card(filtered_df, selected_year, selected_month)
total_savings_card = create_total_savings_card(filtered_df, selected_year, selected_month)

print(spending_fig)
print(savings_fig)
print(total_spending_card)
print(total_savings_card)


{'data': [{'y': ['Automotive', 'Beauty & Personal Care', 'Books', 'Electronics', 'Fashion', 'Grocery', 'Health & Household', 'Home & Kitchen', 'Sports & Outdoors', 'Toys & Games'], 'x': [3881.27, 1393.64, 5281.96, 3562.5, 2384.8, 5051.03, 2511.81, 2795.44, 5314.73, 8995.61], 'type': 'bar', 'orientation': 'h', 'name': 'Spending'}], 'layout': {'title': 'Spending by Category', 'yaxis': {'title': 'Category', 'automargin': True}, 'xaxis': {'title': 'Total Spending'}, 'margin': {'l': 150, 'r': 20, 't': 50, 'b': 50}}}
{'data': [{'y': ['Automotive', 'Beauty & Personal Care', 'Books', 'Electronics', 'Fashion', 'Grocery', 'Health & Household', 'Home & Kitchen', 'Sports & Outdoors', 'Toys & Games'], 'x': [1130.43, 239.62, 1504.69, 233.21999999999997, 1128.0000000000002, 2013.86, 741.9600000000002, 1025.7499999999998, 1752.2299999999998, 3049.5], 'type': 'bar', 'orientation': 'h', 'name': 'Savings'}], 'layout': {'title': 'Savings by Category', 'yaxis': {'title': 'Category', 'automargin': True}, 'x

In [8]:


def plot_spending_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]

    category_spending = filtered_df.groupby('Product Category')['Price for Customer'].sum().reset_index()
    category_spending = category_spending.sort_values(by='Price for Customer', ascending=False)
    
    trace = go.Bar(
        x=category_spending['Price for Customer'],
        y=category_spending['Product Category'],
        orientation='h',
        marker=dict(color='rgba(50, 171, 96, 0.6)'),
        text=category_spending['Price for Customer'].apply(lambda x: f"₹{x:,.2f}"),
        textposition='auto'
    )
    
    layout = go.Layout(
        title='Spending by Category',
        xaxis=dict(title='Total Spending'),
        yaxis=dict(title='Category', automargin=True),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=False
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    return fig

def plot_savings_by_category(filtered_df, selected_year='Overall', selected_month='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Year'] == int(selected_year)]
    if selected_month != 'Overall':
        filtered_df = filtered_df[filtered_df['Order Month'] == selected_month]
    
    category_savings = filtered_df.groupby('Product Category')['Savings'].sum().reset_index()
    category_savings = category_savings.sort_values(by='Savings', ascending=False)
    
    trace = go.Bar(
        x=category_savings['Savings'],
        y=category_savings['Product Category'],
        orientation='h',
        marker=dict(color='rgba(219, 64, 82, 0.6)'),
        text=category_savings['Savings'].apply(lambda x: f"₹{x:,.2f}"),
        textposition='auto'
    )
    
    layout = go.Layout(
        title='Savings by Category',
        xaxis=dict(title='Total Savings'),
        yaxis=dict(title='Category', automargin=True),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=False
    )
    
    fig = go.Figure(data=[trace], layout=layout)
    return fig

filtered_df = df.copy()  
selected_year = 'Overall'
selected_month = 'Overall'

spending_fig = plot_spending_by_category(filtered_df, selected_year=selected_year, selected_month=selected_month)
savings_fig = plot_savings_by_category(filtered_df, selected_year=selected_year, selected_month=selected_month)

spending_fig.show()
print()
savings_fig.show()


In [9]:

def plot_spending_over_months(filtered_df, selected_year='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[pd.to_datetime(filtered_df['Order Date']).dt.year == int(selected_year)]

    monthly_spending = filtered_df.groupby(pd.to_datetime(filtered_df['Order Date']).dt.to_period('M'))['Price for Customer'].sum().reset_index()
    monthly_spending['Date'] = monthly_spending['Order Date'].dt.strftime('%Y-%m')  # Convert to YYYY-MM format
    monthly_spending = monthly_spending.sort_values('Date')

    trace = go.Scatter(
        x=monthly_spending['Date'],
        y=monthly_spending['Price for Customer'],
        mode='lines+markers',
        name='Spending',
        line=dict(color='rgb(50, 171, 96)'),
        marker=dict(symbol='circle', size=8),
        text=monthly_spending['Price for Customer'].apply(lambda x: f"₹{x:,.2f}")
    )

    layout = go.Layout(
        title='Monthly Spending',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Total Spending'),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=True
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

def plot_savings_over_months(filtered_df, selected_year='Overall'):
    if selected_year != 'Overall':
        filtered_df = filtered_df[pd.to_datetime(filtered_df['Order Date']).dt.year == int(selected_year)]

    monthly_savings = filtered_df.groupby(pd.to_datetime(filtered_df['Order Date']).dt.to_period('M'))['Savings'].sum().reset_index()
    monthly_savings['Date'] = monthly_savings['Order Date'].dt.strftime('%Y-%m')  # Convert to YYYY-MM format
    monthly_savings = monthly_savings.sort_values('Date')

    trace = go.Scatter(
        x=monthly_savings['Date'],
        y=monthly_savings['Savings'],
        mode='lines+markers',
        name='Savings',
        line=dict(color='rgb(219, 64, 82)'),
        marker=dict(symbol='circle', size=8),
        text=monthly_savings['Savings'].apply(lambda x: f"₹{x:,.2f}")
    )

    layout = go.Layout(
        title='Monthly Savings',
        xaxis=dict(title='Date'),
        yaxis=dict(title='Total Savings'),
        margin=dict(l=150, r=20, t=50, b=50),
        showlegend=True
    )

    fig = go.Figure(data=[trace], layout=layout)
    return fig

selected_year = '2024'  

spending_over_months_fig = plot_spending_over_months(df, selected_year=selected_year)
savings_over_months_fig = plot_savings_over_months(df, selected_year=selected_year)

spending_over_months_fig.show()
savings_over_months_fig.show()
